In [16]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [25]:
# Load the data 
cardio = pd.read_csv('cardio_base.csv')
alco = pd.read_csv('cardio_alco.csv', sep = ';')
covid_data= pd.read_csv('covid_data.csv')

In [26]:
cardio.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,smoke
0,0,18393,2,168,62.0,110,80,1,0
1,1,20228,1,156,85.0,140,90,3,0
2,2,18857,1,165,64.0,130,70,3,0
3,3,17623,2,169,82.0,150,100,1,0
4,4,17474,1,156,56.0,100,60,1,0


In [19]:
alco.head()

,id,alco
0,44,0
1,45,0
2,46,0
3,47,0
4,49,0


In [20]:
covid_data.head()

,location,date,new_cases,new_deaths,population,aged_65_older_percent,gdp_per_capita,hospital_beds_per_thousand
0,Afghanistan,2019-12-31,0,0,38928341.0,2.581,1803.987,0.5
1,Afghanistan,2020-01-01,0,0,38928341.0,2.581,1803.987,0.5
2,Afghanistan,2020-01-02,0,0,38928341.0,2.581,1803.987,0.5
3,Afghanistan,2020-01-03,0,0,38928341.0,2.581,1803.987,0.5
4,Afghanistan,2020-01-04,0,0,38928341.0,2.581,1803.987,0.5


In [27]:
# Convert age(days) to age in years
cardio['age'] = round(cardio['age'] / 365).astype(int)
cardio

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,smoke
0,0,50,2,168,62.0,110,80,1,0
1,1,55,1,156,85.0,140,90,3,0
2,2,52,1,165,64.0,130,70,3,0
3,3,48,2,169,82.0,150,100,1,0
4,4,48,1,156,56.0,100,60,1,0
...,...,...,...,...,...,...,...,...,...
69995,99993,53,2,168,76.0,120,80,1,1
69996,99995,62,1,158,126.0,140,90,2,0
69997,99996,52,2,183,105.0,180,90,3,0
69998,99998,61,1,163,72.0,135,80,1,0


Do people over 50 have cholestral levels than the rest? 

In [28]:
over_50 = cardio[cardio['age'] > 50]
below_50 = cardio[cardio['age'] <= 50]

print("Descriptive Statistics for Cholesterol Levels - Over 50:")
print(over_50['cholesterol'].describe())

print("\nDescriptive Statistics for Cholesterol Levels - Under 50:")
print(below_50['cholesterol'].describe())

Descriptive Statistics for Cholesterol Levels - Over 50:
count    45984.000000
mean         1.429562
std          0.730529
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          3.000000
Name: cholesterol, dtype: float64

Descriptive Statistics for Cholesterol Levels - Under 50:
count    24016.000000
mean         1.246835
std          0.552275
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          3.000000
Name: cholesterol, dtype: float64


Are men more likely to be a smoker than women?

In [29]:
# Average smoke grouped by gender
smoke_gender = cardio.groupby('gender')['smoke'].mean()
smoke_gender

gender
1    0.017856
2    0.218880
Name: smoke, dtype: float64

In [63]:
# Sum of smokers grouped by gender
g = cardio.groupby('gender').smoke.sum()
g

gender
1     813
2    5356
Name: smoke, dtype: int64

How tall are the tallest 1% of people?

In [64]:
# Calculate 99th Percentile of Height
height_99th_percentile = cardio['height'].quantile(0.99)

print(f"The height of the tallest 1% of people is {height_99th_percentile} units.")


The height of the tallest 1% of people is 184.0 units.


which two features have the highest spearman rank corelation? age and weight,age and blood pressure,blood ressure levels(ap_hi,ap_lo),gender and height

In [30]:

# Compute Spearman Rank Correlation
spearman_corr = cardio.corr(method='spearman')

# Step 3: Identify Pair with Highest Correlation
# Exclude self-correlation (diagonal) and redundant calculations (upper triangle)
spearman_corr = spearman_corr.mask(np.triu(np.ones(spearman_corr.shape)).astype(bool))

# Get the pair with the highest correlation
highest_corr_pair = spearman_corr.abs().stack().idxmax()
highest_corr, feature1, feature2 = spearman_corr.abs().stack().max(), highest_corr_pair[0], highest_corr_pair[1]

print(f"The two features with the highest Spearman rank correlation are '{feature1}' and '{feature2}' with a correlation coefficient of {highest_corr}.")


The two features with the highest Spearman rank correlation are 'ap_lo' and 'ap_hi' with a correlation coefficient of 0.7354361051379777.


what percentage of people are more than 2 standard deviations far from the average height?

In [31]:
mean_height = cardio.height.mean()
std_height = cardio.height.std()
threshold = mean_height + 2 * std_height

# Count Number of People Beyond Threshold
num_people_beyond_threshold = cardio[cardio['height'] > threshold].shape[0]


# Calculate Percentage of People
total_people = cardio.shape[0]
percentage_beyond_threshold = (num_people_beyond_threshold / total_people) * 100

print(f"The percentage of people who are more than 2 standard deviations away from the average height is {percentage_beyond_threshold:.2f}%.")



The percentage of people who are more than 2 standard deviations away from the average height is 2.06%.


what percentage of the population over 50 years old consume alcohol?

In [32]:
# merge the cardio_base.csv and cardio_alco.csv on id 
merge_df = pd.merge(cardio,alco,on = 'id',how = 'left')
merge_df


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,smoke,alco
0,0,50,2,168,62.0,110,80,1,0,NaN
1,1,55,1,156,85.0,140,90,3,0,NaN
2,2,52,1,165,64.0,130,70,3,0,NaN
3,3,48,2,169,82.0,150,100,1,0,NaN
4,4,48,1,156,56.0,100,60,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...
69995,99993,53,2,168,76.0,120,80,1,1,NaN
69996,99995,62,1,158,126.0,140,90,2,0,NaN
69997,99996,52,2,183,105.0,180,90,3,0,NaN
69998,99998,61,1,163,72.0,135,80,1,0,NaN


In [36]:
# Filter for people above 50 and count 
above_50 = merge_df[merge_df['age'] > 50]
above_50

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,smoke,alco
1,1,55,1,156,85.0,140,90,3,0,NaN
2,2,52,1,165,64.0,130,70,3,0,NaN
5,8,60,1,151,67.0,120,80,2,0,NaN
6,9,61,1,157,93.0,130,80,3,0,NaN
7,12,62,2,178,95.0,130,90,3,0,NaN
...,...,...,...,...,...,...,...,...,...,...
69995,99993,53,2,168,76.0,120,80,1,1,NaN
69996,99995,62,1,158,126.0,140,90,2,0,NaN
69997,99996,52,2,183,105.0,180,90,3,0,NaN
69998,99998,61,1,163,72.0,135,80,1,0,NaN


In [38]:
a = above_50[above_50.alco==1].shape[0]
b = above_50[above_50.alco==0].shape[0]
a/(a+b)*100

4.927598297690105

In [ ]:
Smokers have high cholesterol than non smokers

In [117]:
#calculate mean cholesterol where smoke == 1 and smoke == 0
print(np.mean(cardio[cardio['smoke'] == 1].cholesterol))
print(np.mean(cardio[cardio['smoke'] == 0].cholesterol))

1.3895282865942615
1.3646817377136502


In [ ]:
Men have high blood pressure than women

In [124]:
print(np.mean(cardio[cardio.gender==1].ap_hi))
print(np.mean(cardio[cardio.gender== 2].ap_hi))

128.1392488469141
130.07887208827134


In [ ]:
Smokers weigh less than non smokers

In [123]:
print(np.mean(cardio[cardio['smoke'] == 1].weight))
print(np.mean(cardio[cardio['smoke'] == 0].weight))

77.34435078618901
73.90235152198775


Smokers have high blood pressure than non smokers

In [40]:
print(np.mean(cardio[cardio.smoke==1].ap_hi))
print(np.mean(cardio[cardio.smoke==0].ap_hi))

128.36051223861242
128.86143096614498


When did the differrence in the total number of confirmed case between italy and germany become more than 10 000.

In [6]:
covid_data.head()

,location,date,new_cases,new_deaths,population,aged_65_older_percent,gdp_per_capita,hospital_beds_per_thousand
0,Afghanistan,2019-12-31,0,0,38928341.0,2.581,1803.987,0.5
1,Afghanistan,2020-01-01,0,0,38928341.0,2.581,1803.987,0.5
2,Afghanistan,2020-01-02,0,0,38928341.0,2.581,1803.987,0.5
3,Afghanistan,2020-01-03,0,0,38928341.0,2.581,1803.987,0.5
4,Afghanistan,2020-01-04,0,0,38928341.0,2.581,1803.987,0.5


In [41]:

# Filter Data for Italy and Germany
italy_data = covid_data[covid_data['location'] == 'Italy']
germany_data = covid_data[covid_data['location'] == 'Germany']

#Calculate Cumulative Sum of Confirmed Cases
italy_data['cumulative_cases'] = italy_data['new_cases'].cumsum()
germany_data['cumulative_cases'] = germany_data['new_cases'].cumsum()

#merge the two dataframes on date
Italy_germany = pd.merge(italy_data,germany_data,on = 'date',how = 'outer')
Italy_germany


C:\Users\USER\AppData\Local\Temp\ipykernel_6324\1957348205.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  italy_data['cumulative_cases'] = italy_data['new_cases'].cumsum()
C:\Users\USER\AppData\Local\Temp\ipykernel_6324\1957348205.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  germany_data['cumulative_cases'] = germany_data['new_cases'].cumsum()


,location_x,date,new_cases_x,new_deaths_x,population_x,aged_65_older_percent_x,gdp_per_capita_x,hospital_beds_per_thousand_x,cumulative_cases_x,location_y,new_cases_y,new_deaths_y,population_y,aged_65_older_percent_y,gdp_per_capita_y,hospital_beds_per_thousand_y,cumulative_cases_y
0,Italy,2019-12-31,0,0,60461828.0,23.021,35220.084,3.18,0,Germany,0,0,83783945.0,21.453,45229.245,8.0,0
1,Italy,2020-01-01,0,0,60461828.0,23.021,35220.084,3.18,0,Germany,0,0,83783945.0,21.453,45229.245,8.0,0
2,Italy,2020-01-02,0,0,60461828.0,23.021,35220.084,3.18,0,Germany,0,0,83783945.0,21.453,45229.245,8.0,0
3,Italy,2020-01-03,0,0,60461828.0,23.021,35220.084,3.18,0,Germany,0,0,83783945.0,21.453,45229.245,8.0,0
4,Italy,2020-01-04,0,0,60461828.0,23.021,35220.084,3.18,0,Germany,0,0,83783945.0,21.453,45229.245,8.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,Italy,2020-06-06,518,85,60461828.0,23.021,35220.084,3.18,234531,Germany,407,33,83783945.0,21.453,45229.245,8.0,183678
159,Italy,2020-06-07,270,72,60461828.0,23.021,35220.084,3.18,234801,Germany,301,22,83783945.0,21.453,45229.245,8.0,183979
160,Italy,2020-06-08,197,53,60461828.0,23.021,35220.084,3.18,234998,Germany,214,6,83783945.0,21.453,45229.245,8.0,184193
161,Italy,2020-06-09,280,65,60461828.0,23.021,35220.084,3.18,235278,Germany,350,37,83783945.0,21.453,45229.245,8.0,184543


In [42]:
# Calculate the difference in the cumulative cases and fiter where the difference is > 10000
Italy_germany['diff'] = Italy_germany.cumulative_cases_x - Italy_germany.cumulative_cases_y
Italy_germany = Italy_germany[Italy_germany['diff'] > 10000]
Italy_germany

,location_x,date,new_cases_x,new_deaths_x,population_x,aged_65_older_percent_x,gdp_per_capita_x,hospital_beds_per_thousand_x,cumulative_cases_x,location_y,new_cases_y,new_deaths_y,population_y,aged_65_older_percent_y,gdp_per_capita_y,hospital_beds_per_thousand_y,cumulative_cases_y,diff
72,Italy,2020-03-12,2313,196,60461828.0,23.021,35220.084,3.18,12462,Germany,271,1,83783945.0,21.453,45229.245,8.0,1567,10895
73,Italy,2020-03-13,2651,189,60461828.0,23.021,35220.084,3.18,15113,Germany,802,2,83783945.0,21.453,45229.245,8.0,2369,12744
74,Italy,2020-03-14,2547,252,60461828.0,23.021,35220.084,3.18,17660,Germany,693,0,83783945.0,21.453,45229.245,8.0,3062,14598
75,Italy,2020-03-15,3497,173,60461828.0,23.021,35220.084,3.18,21157,Germany,733,3,83783945.0,21.453,45229.245,8.0,3795,17362
76,Italy,2020-03-16,2823,370,60461828.0,23.021,35220.084,3.18,23980,Germany,1043,4,83783945.0,21.453,45229.245,8.0,4838,19142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,Italy,2020-06-06,518,85,60461828.0,23.021,35220.084,3.18,234531,Germany,407,33,83783945.0,21.453,45229.245,8.0,183678,50853
159,Italy,2020-06-07,270,72,60461828.0,23.021,35220.084,3.18,234801,Germany,301,22,83783945.0,21.453,45229.245,8.0,183979,50822
160,Italy,2020-06-08,197,53,60461828.0,23.021,35220.084,3.18,234998,Germany,214,6,83783945.0,21.453,45229.245,8.0,184193,50805
161,Italy,2020-06-09,280,65,60461828.0,23.021,35220.084,3.18,235278,Germany,350,37,83783945.0,21.453,45229.245,8.0,184543,50735


which country had the 3rd highest death rate? Death rate = total number death per millon inhabitants 

In [46]:
#groups the covid_data DataFrame by the "location" column, then calculates the sum of the "new_deaths" column for each group. 
#It creates a new DataFrame temp with the summed values for each location
temp = pd.DataFrame(covid_data.groupby("location")["new_deaths"].sum())
temp = temp.sort_values(by="new_deaths", ascending=False)
print(temp)


                                 new_deaths
location                                   
World                                411195
United States                        112006
United Kingdom                        40883
Brazil                                38406
Italy                                 34043
...                                     ...
Bonaire Sint Eustatius and Saba           0
Seychelles                                0
Bhutan                                    0
Saint Kitts and Nevis                     0
Dominica                                  0

[212 rows x 1 columns]


In [47]:
#calculate the mortality rate (new deaths divide by population) for each location
pop = []
for i in covid_data.location.unique():
    pop.append((temp[temp.index==i].new_deaths.tolist()[0]/covid_data.groupby("location").get_group(i)["population"].tolist()[0],i))
pop.sort(reverse=True)
pop

[(0.0012375508279804349, 'San Marino'),
 (0.0008299671015847289, 'Belgium'),
 (0.0006600660066006601, 'Andorra'),
 (0.0006022301739840218, 'United Kingdom'),
 (0.0005803898180855635, 'Spain'),
 (0.0005630494665162952, 'Italy'),
 (0.00046706346102243034, 'Sweden'),
 (0.00044881911670387826, 'France'),
 (0.0003519722614810159, 'Netherlands'),
 (0.0003497971176717504, 'Sint Maarten (Dutch part)'),
 (0.00034246048236905697, 'Ireland'),
 (0.00033838400089894146, 'United States'),
 (0.00029681517319165353, 'Jersey'),
 (0.0002822466836014677, 'Isle of Man'),
 (0.00020923552408517616, 'Canada'),
 (0.0002091473927992083, 'Ecuador'),
 (0.00020004000800160032, 'Montserrat'),
 (0.00019387937719978523, 'Guernsey'),
 (0.00019203620541079919, 'Switzerland'),
 (0.00018068360361314328, 'Brazil'),
 (0.00017572558692346033, 'Luxembourg'),
 (0.00017402725949890703, 'Peru'),
 (0.00014632174877634514, 'Portugal'),
 (0.0001445249144894256, 'Bermuda'),
 (0.00012740801141575782, 'Monaco'),
 (0.0001194274450546

Countries where more than 20% of the population is over 65 years old, have death rates over 50 per milion inhabitants 

In [50]:
# Calculate the mortality rate for each location in the COVID-19 dataset,where 'aged_65_older_percent' is greater than 20%.
pop = []
for i in covid_data.location.unique():
    if covid_data.groupby("location").get_group(i)["aged_65_older_percent"].tolist()[0]>20:
        pop.append((temp[temp.index==i].new_deaths.tolist()[0]/covid_data.groupby("location").get_group(i)["population"].tolist()[0],
        covid_data.groupby("location").get_group(i)["aged_65_older_percent"].tolist()[0],i))
pop.sort(reverse=True)
pop

[(0.0005630494665162952, 23.021, 'Italy'),
 (0.00014632174877634514, 21.502, 'Portugal'),
 (0.00010418463823826868, 21.453, 'Germany'),
 (5.847617583136337e-05, 21.228, 'Finland'),
 (2.4034154404330752e-05, 20.801, 'Bulgaria'),
 (1.755723081599101e-05, 20.396, 'Greece'),
 (7.266174389545286e-06, 27.049, 'Japan')]

what is the probabilty taht a country has a gdp over $ 10,000 ,if we know that they have at least 5 hospital beds per 1000 inhabitants

In [51]:
# mx counts the number of countries where the number of hospital beds per thousand inhabitants is greater than or equal to 5
# mn counts the number of countries where it is less than 5.
#pop = [] Initializes an empty list pop where we will store the results of our analysis.
mx = 0
mn = 0
pop = []
for i in covid_data.location.unique():
    if covid_data.groupby("location").get_group(i)["gdp_per_capita"].tolist()[0]>10000:
        pop.append((covid_data.groupby("location").get_group(i)["hospital_beds_per_thousand"].tolist()[0],covid_data.groupby("location").get_group(i)["gdp_per_capita"].tolist()[0],i))
        if covid_data.groupby("location").get_group(i)["hospital_beds_per_thousand"].tolist()[0]>=5:
            mx+=1
        else:
            mn+=1
mx/(mx+mn)

0.21904761904761905